In [1]:
import os
import json
import numpy
import datetime
import requests 
import pandas as pd

import pymongo
from sqlalchemy import create_engine

In [2]:

mysql_uid = "gab8un"
mysql_pwd = "Passw0rd123"
mysql_url = "mysql-gab8un.mysql.database.azure.com"

atlas_cluster_name = "cluster0.q0atcnd"
atlas_user_name = "gab8un"
atlas_password = "XwlxqfpWD8PxodaA"

mongo_connections = {
    "local": f"mongodb://localhost:27017/",
    "atlas": f"mongodb+srv://{atlas_user_name}:{atlas_password}@{atlas_cluster_name}.mongodb.net"
}

conn_str = mongo_connections['atlas']
source_db = 'sakila'
dest_db = 'sakila_dw'

In [3]:

def get_sql_dataframe(user_id, pwd, db_url, db_name, sql_query):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{db_url}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    
    '''Invoke the pd.read_sql() function to query the database, and fill a Pandas DataFrame.'''
    conn = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, conn);
    conn.close()
    
    return dframe


def get_mongo_dataframe(connect_str, db_name, collection, query):
    '''Create a connection to MongoDB'''
    client = pymongo.MongoClient(connect_str)
    
    '''Query MongoDB, and fill a python list with documents to create a DataFrame'''
    db = client[db_name]
    dframe = pd.DataFrame(list(db[collection].find(query)))
    dframe.drop(['_id'], axis=1, inplace=True)
    client.close()
    return dframe

def execute_sql(user_id, pwd, db_url, sql):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{db_url}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    connection.execute(sql)
    connection.close()



def set_dataframe(user_id, pwd, db_url, db_name, df, table_name, pk_column, db_operation):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{db_url}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the Pandas DataFrame .to_sql( ) function to either create, or append to, a table'''
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        connection.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});")
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()

## Load data from database

The majority of the data that is used comes from the MySQL database


In [4]:
rentals_sql = "SELECT r.*, p.amount FROM sakila.rental AS r INNER JOIN sakila.payment AS p ON r.rental_id = p.rental_id;"
rentals_df = get_sql_dataframe(mysql_uid, mysql_pwd, mysql_url, source_db, rentals_sql)
rentals_df.head(5)

rental_id         rental_date  inventory_id  customer_id  \
0          1 2005-05-24 22:53:30           367          130   
1          2 2005-05-24 22:54:33          1525          459   
2          3 2005-05-24 23:03:39          1711          408   
3          4 2005-05-24 23:04:41          2452          333   
4          5 2005-05-24 23:05:21          2079          222   

          return_date  staff_id         last_update  amount  
0 2005-05-26 22:04:30         1 2006-02-15 21:30:53    2.99  
1 2005-05-28 19:40:33         1 2006-02-15 21:30:53    2.99  
2 2005-06-01 22:12:39         1 2006-02-15 21:30:53    3.99  
3 2005-06-03 01:43:41         2 2006-02-15 21:30:53    4.99  
4 2005-06-02 04:33:21         1 2006-02-15 21:30:53    6.99

In [5]:
staff_sql = "SELECT * FROM sakila.staff;"
staff_df = get_sql_dataframe(mysql_uid, mysql_pwd, mysql_url, source_db, staff_sql)
staff_df.head(5)

staff_id first_name last_name  address_id  \
0         1       Mike   Hillyer           3   
1         2        Jon  Stephens           4   

                                             picture  \
0  b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...   
1                                               None   

                          email  store_id  active username  \
0  Mike.Hillyer@sakilastaff.com         1       1     Mike   
1  Jon.Stephens@sakilastaff.com         2       1      Jon   

                                   password         last_update  
0  8cb2237d0679ca88db6464eac60da96345513964 2006-02-15 03:57:16  
1                                      None 2006-02-15 03:57:16

In [6]:
stores_sql = "SELECT store_id, manager_staff_id, address FROM sakila.store AS s INNER JOIN sakila.address as a ON s.address_id = a.address_id;"
stores_df = get_sql_dataframe(mysql_uid, mysql_pwd, mysql_url, source_db, stores_sql)
stores_df.head(5)

store_id  manager_staff_id             address
0         1                 1   47 MySakila Drive
1         2                 2  28 MySQL Boulevard

In [7]:
payment_sql = "SELECT * FROM sakila.payment";
payment_df = get_sql_dataframe(mysql_uid, mysql_pwd, mysql_url, source_db, payment_sql)
payment_df.head(5)

payment_id  customer_id  staff_id  rental_id  amount        payment_date  \
0           1            1         1         76    2.99 2005-05-25 11:30:37   
1           2            1         1        573    0.99 2005-05-28 10:35:23   
2           3            1         1       1185    5.99 2005-06-15 00:54:12   
3           4            1         2       1422    0.99 2005-06-15 18:02:53   
4           5            1         2       1476    9.99 2005-06-15 21:08:46   

          last_update  
0 2006-02-15 22:12:30  
1 2006-02-15 22:12:30  
2 2006-02-15 22:12:30  
3 2006-02-15 22:12:30  
4 2006-02-15 22:12:30

In [8]:
inventory_sql = "SELECT * FROM sakila.inventory;"
inventory_df = get_sql_dataframe(mysql_uid, mysql_pwd, mysql_url, source_db, inventory_sql)
inventory_df.head(5)

inventory_id  film_id  store_id         last_update
0             1        1         1 2006-02-15 05:09:17
1             2        1         1 2006-02-15 05:09:17
2             3        1         1 2006-02-15 05:09:17
3             4        1         1 2006-02-15 05:09:17
4             5        1         2 2006-02-15 05:09:17

## Retrieve data from remote sources
Data is stored in a mongoDB database and at an API endpoint

In [9]:
film_api_endpoint = "https://api.npoint.io/04c66e9121c9572e6442"
film_df = pd.read_json(film_api_endpoint)
film_df.head(5)

title  length rating  film_id  \
0  ACADEMY DINOSAUR      86     PG        1   
1    ACE GOLDFINGER      48      G        2   
2  ADAPTATION HOLES      50  NC-17        3   
3  AFFAIR PREJUDICE     117      G        4   
4       AFRICAN EGG     130      G        5   

                                         description  language_id  \
0  A Epic Drama of a Feminist And a Mad Scientist...            1   
1  A Astounding Epistle of a Database Administrat...            1   
2  A Astounding Reflection of a Lumberjack And a ...            1   
3  A Fanciful Documentary of a Frisbee And a Lumb...            1   
4  A Fast-Paced Documentary of a Pastry Chef And ...            1   

     last_update  rental_rate  release_year  rental_duration  \
0  1139979822000         0.99          2006                6   
1  1139979822000         4.99          2006                3   
2  1139979822000         2.99          2006                7   
3  1139979822000         2.99          2006                5   
4  1139979822000         2.99          2006                6   

   replacement_cost                  special_features  original_language_id  
0             20.99  Deleted Scenes,Behind the Scenes                   NaN  
1             12.99           Trailers,Deleted Scenes                   NaN  
2             18.99           Trailers,Deleted Scenes                   NaN  
3             26.99    Commentaries,Behind the Scenes                   NaN  
4             22.99                    Deleted Scenes                   NaN

In [10]:
query = {}
collection = "customers"
customers_df= get_mongo_dataframe(conn_str, source_db, collection, query)
customers_df.head(5)

customer_id  store_id first_name last_name  \
0            1         1       MARY     SMITH   
1            2         1   PATRICIA   JOHNSON   
2            3         1      LINDA  WILLIAMS   
3            4         2    BARBARA     JONES   
4            5         1  ELIZABETH     BROWN   

                                 email  address_id  active    create_date  \
0        MARY.SMITH@sakilacustomer.org           5       1  1139954676000   
1  PATRICIA.JOHNSON@sakilacustomer.org           6       1  1139954676000   
2    LINDA.WILLIAMS@sakilacustomer.org           7       1  1139954676000   
3     BARBARA.JONES@sakilacustomer.org           8       1  1139954676000   
4   ELIZABETH.BROWN@sakilacustomer.org           9       1  1139954676000   

     last_update  
0  1139979440000  
1  1139979440000  
2  1139979440000  
3  1139979440000  
4  1139979440000

## Perform Transformations

In [11]:
dim_staff_df = staff_df.drop(['address_id', 'picture', 'store_id', 'active', 'username', 'password', 'last_update'], axis=1)
dim_staff_df.rename(columns={"staff_id": "staff_key"}, inplace=True)
dim_staff_df.head(2)

staff_key first_name last_name                         email
0          1       Mike   Hillyer  Mike.Hillyer@sakilastaff.com
1          2        Jon  Stephens  Jon.Stephens@sakilastaff.com

In [12]:
dim_stores_df = stores_df.rename(columns={"store_id": "store_key", "manager_staff_id": "staff_key"})
dim_stores_df.fact_stores_df = pd.merge(dim_stores_df, dim_staff_df, on="staff_key", how='inner')
dim_stores_df.fact_stores_df.drop(['staff_key'], axis=1, inplace=True)
dim_stores_df.rename(columns={
    "first_name": "manager_first_name", 
    "last_name": "manager_first_name",
    "email": "manager_email"
}, inplace=True)
dim_stores_df.head()

/tmp/ipykernel_54860/2457610734.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  dim_stores_df.fact_stores_df = pd.merge(dim_stores_df, dim_staff_df, on="staff_key", how='inner')


store_key  staff_key             address
0          1          1   47 MySakila Drive
1          2          2  28 MySQL Boulevard

In [13]:
dim_customers_df = customers_df.rename(columns={"customer_id":"customer_key"})
dim_customers_df.drop([
    "store_id",
    "address_id", 
    "active",
    "create_date", 
    "last_update",
], axis=1, inplace=True)
dim_customers_df.head(2)

customer_key first_name last_name                                email
0             1       MARY     SMITH        MARY.SMITH@sakilacustomer.org
1             2   PATRICIA   JOHNSON  PATRICIA.JOHNSON@sakilacustomer.org

In [14]:
dim_films_df = film_df.rename(columns={"film_id":"film_key"})
dim_films_df.drop([
    "description", 
    "last_update",
    "rental_rate", 
    "special_features", 
    "original_language_id",
    "language_id"
], axis=1, inplace=True)
dim_films_df.head(2)

title  length rating  film_key  release_year  rental_duration  \
0  ACADEMY DINOSAUR      86     PG         1          2006                6   
1    ACE GOLDFINGER      48      G         2          2006                3   

   replacement_cost  
0             20.99  
1             12.99

In [24]:
fact_rentals_df = pd.merge(rentals_df, inventory_df, on='inventory_id', how='inner')
fact_rentals_df.drop(['inventory_id', 'last_update_x', 'last_update_y'], axis=1, inplace=True)
fact_rentals_df.rename(columns={
    "rental_id": "fact_rental_key", 
    "film_id": "film_key",
    "staff_id": "staff_key",
    "store_id": "store_key",
    "customer_id": "customer_key",
}, inplace=True)
order=['fact_rental_key', 'film_key', 'staff_key', 'store_key', 'customer_key', 'rental_date', 'return_date', 'amount']
fact_rentals_df = fact_rentals_df[order]
fact_rentals_df.sort_values(by=['fact_rental_key'], inplace=True)
fact_rentals_df.reset_index(drop=True, inplace=True)
fact_rentals_df.head(200)

fact_rental_key  film_key  staff_key  store_key  customer_key  \
0                  1        80          1          1           130   
1                  2       333          1          2           459   
2                  3       373          1          2           408   
3                  4       535          2          1           333   
4                  5       450          1          2           222   
..               ...       ...        ...        ...           ...   
195              196        43          1          1           184   
196              197       915          2          2           546   
197              198       388          1          2            54   
198              199       462          1          1            71   
199              200       856          1          2           321   

            rental_date         return_date  amount  
0   2005-05-24 22:53:30 2005-05-26 22:04:30    2.99  
1   2005-05-24 22:54:33 2005-05-28 19:40:33    2.99  
2   2005-05-24 23:03:39 2005-06-01 22:12:39    3.99  
3   2005-05-24 23:04:41 2005-06-03 01:43:41    4.99  
4   2005-05-24 23:05:21 2005-06-02 04:33:21    6.99  
..                  ...                 ...     ...  
195 2005-05-26 06:55:58 2005-05-27 10:54:58    2.99  
196 2005-05-26 06:59:21 2005-06-03 05:04:21    5.99  
197 2005-05-26 07:03:49 2005-06-04 11:45:49    4.99  
198 2005-05-26 07:11:58 2005-05-28 09:06:58    2.99  
199 2005-05-26 07:12:21 2005-05-31 12:07:21    4.99  

[200 rows x 8 columns]

## Load the data into the data warehouse

In [16]:
# Drop the database if it exists
drop_db_sql = f"DROP SCHEMA IF EXISTS {dest_db}"
execute_sql(mysql_uid, mysql_pwd, mysql_url, drop_db_sql)

# Create the empty database
create_db_sql = f"CREATE SCHEMA IF NOT EXISTS {dest_db}"
execute_sql(mysql_uid, mysql_pwd, mysql_url, create_db_sql)

In [17]:
table_name = 'dim_customers'
primary_key = 'customer_key'
db_operation = 'insert'
table = dim_customers_df
set_dataframe(mysql_uid, mysql_pwd, mysql_url, dest_db, table, table_name, primary_key, db_operation )

In [18]:

table_name = 'dim_films'
primary_key = 'film_key'
db_operation = 'insert'
table = dim_films_df 
set_dataframe(mysql_uid, mysql_pwd, mysql_url, dest_db, table, table_name, primary_key, db_operation )

In [19]:

table_name = 'dim_staff'
primary_key = 'staff_key'
db_operation = 'insert'
table = dim_staff_df 
set_dataframe(mysql_uid, mysql_pwd, mysql_url, dest_db, table, table_name, primary_key, db_operation )

In [20]:
table_name = 'dim_stores'
primary_key = 'store_key'
db_operation = 'insert'
table = dim_stores_df 
set_dataframe(mysql_uid, mysql_pwd, mysql_url, dest_db, table, table_name, primary_key, db_operation )

In [25]:
table_name = 'fact_rentals'
primary_key = 'fact_rental_key'
db_operation = 'insert'
table = fact_rentals_df 
set_dataframe(mysql_uid, mysql_pwd, mysql_url, dest_db, table, table_name, primary_key, db_operation )

## Ensure the data has been loaded into the database

In [32]:
# Check how many rentals each employee has handled 
rental_count_sql = """
SELECT staff.first_name, staff.last_name, COUNT(rentals.fact_rental_key) AS rental_count FROM sakila_dw.fact_rentals AS rentals
INNER JOIN sakila_dw.dim_staff AS staff
ON rentals.staff_key = staff.staff_key
GROUP BY staff.first_name, staff.last_name;
"""
rental_count_df = get_sql_dataframe(mysql_uid, mysql_pwd, mysql_url, dest_db, rental_count_sql)
rental_count_df.head()

first_name last_name  rental_count
0        Jon  Stephens          8004
1       Mike   Hillyer          8040

In [33]:
# Get the top 100 most rented movies
top_movies_sql = """ 
SELECT film.title, COUNT(rental.fact_rental_key) AS rental_count FROM sakila_dw.fact_rentals AS rental
INNER JOIN sakila_dw.dim_films AS film
ON rental.film_key = film.film_key
GROUP BY film.title
ORDER BY COUNT(rental.fact_rental_key) DESC
LIMIT 100;
"""
top_movies_df = get_sql_dataframe(mysql_uid, mysql_pwd, mysql_url, dest_db, top_movies_sql)
top_movies_df


title  rental_count
0      BUCKET BROTHERHOOD            34
1        ROCKETEER MOTHER            33
2          GRIT CLOCKWORK            32
3     RIDGEMONT SUBMARINE            32
4          JUGGLER HARDLY            32
..                    ...           ...
95  STAGECOACH ARMAGEDDON            26
96          MALKOVICH PET            26
97    BROTHERHOOD BLANKET            26
98      FELLOWSHIP AUTUMN            26
99      VELVET TERMINATOR            26

[100 rows x 2 columns]

In [34]:
# Get total rental profit
total_revenue_sql = "SELECT SUM(amount) AS total_profit from sakila_dw.fact_rentals;"
total_revenue_df = get_sql_dataframe(mysql_uid, mysql_pwd, mysql_url, dest_db, total_revenue_sql)
total_revenue_df.head()

total_profit
0      67406.56